In [ ]:
import time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pandas as pd

In [2]:
# 4130 page까지임 

for helper in range(1, 43):
#for helper in range(1, 2):  ###test
    driver = webdriver.Chrome("./chromedriver")

    titles = []
    paths = []
    contents = []
    dates = []
    nums = []
    url = ""

    i = (100 * (helper - 1)) + 1
    #원래 => for page in range(i, 100+i):
    for page in range(i, 100+i):
    #for page in range(i, 1+i): ###test
        if page == 4131: #page는 4130 까지밖에 없음.
            break
        url = 'https://www1.president.go.kr/petitions/?c=42&only=2&page={}&order=1'.format(page)
        driver.get(url)
        print("current page : ", page)

        # driver.get(url)로 서버에게 get 요청을 보냈으니 response를 기다려야함. html을 받는데 걸리는 최소시간 -> 1.5s
        time.sleep(1.5)
        soup = bs(driver.page_source, 'html.parser') # 3초가 끝나면 새로운 page의 url로 접속하고 파서가 파싱한다.

        # 현 page에서 petition 요소에 접근
        lists = soup.find("div", attrs={"class": "ct_list1"})
        ul = lists.find("ul", attrs={"class": "petition_list"})
        a_lists = ul.find_all("a", attrs={"class": "cb relpy_w"})  # 현재 페이지의 "전체목록" 내부에 있는 모든 a 태그들을 리스트로 반환.

        # path를 추가하기 전에 현재 paths 리스트에 몇개가 들어있는지 확인한다 => 2 page가 되면 paths 리스트의 7번 index부터 읽어야하기 때문
        path_len = len(paths)
        for a in a_lists: # 현재 page 내에서 각 petition의 title과 path를 파싱하여 paths, titles 배열에 저장.
            # print(a.attrs["href"]) path 배열에 들어갈 경로 "/petition/12345"
            paths.append(a.attrs["href"])  # 각 청원의 path를 저장.
            # print(a.text[3:].strip()) titles
            titles.append(a.text[3:].strip())  # 각 청원의 title을 titles 배열에 저장.

        new_paths = paths[path_len:] # page 별로 해당하는 path만 읽어야하므로 => 2 page가 되면 paths 리스트의 7번 index부터 읽어야하기 때문
        print("new_paths : ", new_paths)
        for path in new_paths: # 현재 page의 모든 petition들의 해당 url에 접속하여 content를 파싱.
            url = "https://www1.president.go.kr" + path
            driver.get(url)
            soup = bs(driver.page_source, 'html.parser') # 각 url에 대하여 soup 객체를 새로 생성한다.

            # contents 추가하기.
            content = soup.find("div", attrs={"class":"View_write"})
            data = content.text.strip()
            data = data.replace("\n", "") # \n 문자도 제거
            #print('본문 내용 : ', data)

            # date 추가하기.
            date_ul = soup.find("ul", attrs={"class": "petitionsView_info_list"})
            li = date_ul.li.next_sibling.next_sibling
            li_data = li.text[4:]

            # num 추가하기.
            h2 = soup.find("h2", attrs={"class": "petitionsView_count"})
            num = h2.find('span', attrs={"class": "counter"})
            num = num.text

            contents.append(data) # 공백 제거한 data를 리스트에 저장.
            dates.append(li_data)
            nums.append(num)
    driver.close()
    # 최종 값
    print("titles : ", titles)
    print("nums 배열의 수로 확인하기 ", len(nums)) # num의 갯수가 700개(100페이지 * 7개 청원글)여야 안뺴놓고 크롤링 한거임.
    #print("contents : ", contents)
    print("dates : ", dates)
    print("nums : ", nums)

    my_dictionary = {"title": titles, "date": dates, "num": nums, "contents": contents}
    data = pd.DataFrame(my_dictionary)  # 전체 데이터를 긁은 list인 total을 dataframe으로 변환시켜주면서 각 column의 이름을 부여해줍니다.

    str_i = str(i)
    str_i2 = str(i+99)
    datatitle = str_i + '_' + str_i2
    # datatitle = input("Please set the title of this excel file : ")  # 긁어온 데이터를 저장할 xlsx 파일의 이름을 지정해줍니다.
    data.to_excel("crawling_data/" + datatitle + ".xlsx", engine="xlsxwriter")
    #xlsxwriter를 임포트시켜서 engine을 추가하지 않으면, data.to_excel에서 IllegalCharacterError 에러가 뜬다.

current page :  1
new_paths :  ['/petitions/596284', '/petitions/596268', '/petitions/596267', '/petitions/596248', '/petitions/596239', '/petitions/596220', '/petitions/596215']
current page :  2
new_paths :  ['/petitions/596210', '/petitions/596186', '/petitions/596181', '/petitions/596160', '/petitions/596153', '/petitions/596139', '/petitions/596108']
current page :  3
new_paths :  ['/petitions/596101', '/petitions/596092', '/petitions/596087', '/petitions/596083', '/petitions/596077', '/petitions/596072', '/petitions/596063']
current page :  4
new_paths :  ['/petitions/596052', '/petitions/596046', '/petitions/596042', '/petitions/596039', '/petitions/596010', '/petitions/596001', '/petitions/596000']
current page :  5
new_paths :  ['/petitions/595993', '/petitions/595990', '/petitions/595988', '/petitions/595986', '/petitions/595975', '/petitions/595972', '/petitions/595951']
current page :  6
new_paths :  ['/petitions/595947', '/petitions/595940', '/petitions/595938', '/petition

C:\Users\uni61\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:935: UserWarning: Ignoring URL 'https://blog.naver.com/lawhaeyul/***************-%20%5b추가%5d%20산후도우미%20‘생후25일’%20신생아학대%20원본영상https://blog.naver.com/lawhaeyul/***************-%20‘생후%2025일’%20신생아%20학대한%20산후도우미%20영장실질심사%20/%20MBN뉴스국민의%20고충을%20위해%20언제나%20노력해주시는%20대통령님과%20청와대%20관계자분들,이%20청원을%20읽어주시는%20모든%20분들에게%20감사드리고,피해%20아기와%20가족들이%20받은%20고통과%20현재까지%20겪고%20있는%20고통을%20생각해주시어위의%20해당%20링크를%20꼭%20한%20번%20봐주시고%20이%20글을%20읽어주시길%20간곡히%20부탁드립니다.%20%20세상에%20태어난%20지%2025일%20밖에%20되지%20않은%20아기가%20있습니다..%20%20그%20아이는%20심장판막이%20제대로%20형성되지%20못한%20채%20태어났습니다...%20%20생후%2025일밖에%20안%20된%20**(가명)는%20아직%20몸에%20뼈도%20여물지%20않았을%20것이고,%20색깔%20구분도%20어려운%20시기라%20흑백만%20볼%20수%20있다고%20합니다.%20신체의%20모든%20기관이%20극도로%20예민하며,%20태어나서%20가장%20중요한%20시기입니다.%20%20%20게다가%20**는%20심장이%20워낙%20약하여%20다른%20아기보다%20더욱%20더%20충격과%20스트레스에%20주의를%20기울여야합니다.%20하지만%20그런%20**는,%20%20‘할머니%20하지마세요.%20너무%20아파요.%20고통스러워요...’%20%20등의%20말도%20못하고,%20아무런%20감정표현도%20할%20수%20없었던%20아기는...%2

current page :  201
new_paths :  ['/petitions/581891', '/petitions/581888', '/petitions/581823', '/petitions/581820', '/petitions/581809', '/petitions/581786', '/petitions/581752']
current page :  202
new_paths :  ['/petitions/581740', '/petitions/581726', '/petitions/581721', '/petitions/581718', '/petitions/581711', '/petitions/581687', '/petitions/581686']
current page :  203
new_paths :  ['/petitions/581682', '/petitions/581679', '/petitions/581678', '/petitions/581641', '/petitions/581626', '/petitions/581604', '/petitions/581562']
current page :  204
new_paths :  ['/petitions/581560', '/petitions/581535', '/petitions/581525', '/petitions/581521', '/petitions/581510', '/petitions/581502', '/petitions/581500']
current page :  205
new_paths :  ['/petitions/581463', '/petitions/581458', '/petitions/581456', '/petitions/581441', '/petitions/581435', '/petitions/581433', '/petitions/581421']
current page :  206
new_paths :  ['/petitions/581412', '/petitions/581410', '/petitions/581405'

C:\Users\uni61\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:935: UserWarning: Ignoring URL 'https://www.facebook.com/incheonmessage2/videos/1886856504752200/사건%20동영상입니다.1차폭행만%20포착이되었습니다.https://www1.president.go.kr/petitions/555878?navigation=petitions요기%20링크로국민청원%20동의부탁드립니다SNS힘을%20빌려서%20장애인폭행을%20아무렇지않게%20생각하지않는%20가해자에게%20엄중한%20처벌을%20부탁드립니다.인천에%20한%20Y사%20사찰에서일어난%20일입니다이틀전에제동생이%20묻지마폭행을당해서%20지금%20굉장히%20심리적인문제와%20상해때문에%20고생하고있습니다.사건이%20일어난이유가%20절에서%20앉아있는데%20어떤30대주부가%20제동생에게%20안좋은기운이있어서%20장애인이%20여기절에있으면%20안된다며%20갑자기%20구타가시작되었는데%20모든사람들이%20말렸지만%20계속구타가%20일어나는중에%20경찰출동%20후%20멈춰졌습니다.왜때렸냐고물어보니그%20절에%20총책임자인%20(주지스님%20불러와)%20라면서이장애인은%20이절에%20오면%20안된다면서부적?같은걸%20태웠다고%20말도안되는%20얘기를하면서%20지구대경찰애들이%20제동생한테라이터%20있냐고%20뒤져보게되었고%20제동생은%20지적장애1급이며%20라이터켜는법도모릅니다.이렇게되서%20경찰서에%20조서를쓰고%20다음날그사람이또%20범행동기가%20제%20동생이%20바지를내리고%20성기를드러냈었고%20수시로%20안좋은%20문자로%20치근덕댔다고%20참지못해폭발하여%20폭행했다고합니다.처음%20연행되고%20진술%20당시%20담당형사가%20혐의가100프로인정되기때문에%20영상확인할필요없이%20가만히계시면된다고%20하더니%20다음날%20갑자기%20그런이유로%20말도안되게%20몰아부쳤고%20제동생이%

current page :  401
new_paths :  ['/petitions/540014', '/petitions/539999', '/petitions/539998', '/petitions/539924', '/petitions/539903', '/petitions/539893', '/petitions/539891']
current page :  402
new_paths :  ['/petitions/539870', '/petitions/539853', '/petitions/539852', '/petitions/539813', '/petitions/539798', '/petitions/539739', '/petitions/539736']
current page :  403
new_paths :  ['/petitions/539718', '/petitions/539709', '/petitions/539671', '/petitions/539604', '/petitions/539414', '/petitions/539405', '/petitions/539360']
current page :  404
new_paths :  ['/petitions/539353', '/petitions/539352', '/petitions/539337', '/petitions/539308', '/petitions/539303', '/petitions/539272', '/petitions/539263']
current page :  405
new_paths :  ['/petitions/539242', '/petitions/539229', '/petitions/539219', '/petitions/539213', '/petitions/539192', '/petitions/539178', '/petitions/539177']
current page :  406
new_paths :  ['/petitions/539145', '/petitions/539134', '/petitions/539063'

KeyboardInterrupt: 